In [2]:
import numpy as np
import pandas as pd
import os

In [7]:
path_of_recordings = str(os.getcwd()) + '/recordings/'

In [8]:
path_of_recordings

'/Users/Anonymous/Desktop/OCET/recordings/'

In [9]:
for r, d, f in os.walk(path_of_recordings):
    for file in f:
        print(os.path.join(r, file))

/Users/Anonymous/Desktop/OCET/recordings/.DS_Store
/Users/Anonymous/Desktop/OCET/recordings/recording_file_a.txt
